In [22]:
from osgeo import gdal, osr
from osgeo import osr
import geopandas as gpd
from osgeo import ogr
from shapely.geometry import Polygon, Point
from geopandas.geoseries import *

In [23]:
import tarfile
import pandas as pd
import datetime as dt
import os
import glob
import subprocess
import os.path
from os import path
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from terrautils.spatial import scanalyzer_to_latlon

In [24]:
# Grabs the thermal raw data tar file for the Gantry's metadata for the images taken at the specified date
class JSON:
    def get_tar(season, date):
        command = f'iget -rKTPf -N 0 /iplant/home/shared/terraref/ua-mac/raw_tars/{season}/flirIrCamera/flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)
        command = f'tar -xvf flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)

# Finds the individual json files and adds them to the filepath (end up with a list of paths for each individual json file)
    def pathlist(season, date):
        json_data = JSON.get_tar(season, date)
        pathlist = Path(f"./flirIrCamera/{date}/").glob('**/*.json')
        JSON_path_list = []
        for path in pathlist:
            path_str = str(path)
            JSON_path_list.append(path_str)
        return JSON_path_list

# Uses the pathlists and searches each one
# Gathers the time, image name, gantry_x and gantry_y information from the metadata of each individual image and adds to dictionary
    def time_dict(season, date):
        file_path_list = JSON.pathlist(season, date)
        JSON_dict = dict()
        for file in file_path_list:
            path_metadata = glob.glob(f'{file}')
            metadata = str(path_metadata)[2:-2]
            with open(metadata) as f:
                meta = json.load(f)['lemnatec_measurement_metadata']
                time = (meta['gantry_system_variable_metadata']['time'])
                gantry_x = float(meta['gantry_system_variable_metadata']['position x [m]'])
                gantry_y = float(meta['gantry_system_variable_metadata']['position y [m]'])
                filename = os.path.basename(metadata)
            if JSON is not JSON_dict:
                JSON_dict[time, filename, gantry_x, gantry_y] = "Date, Time, Gantry_x, Gantry_y"
            else:
                print("JSON already in Dictionary")
        return sorted(JSON_dict)

# Searches through the dictionary created and creates a dataframe of the information in the dictionary
    def time_df(season, date):
        JSON_time_d = JSON.time_dict(season, date)
        JSON_time_df = pd.DataFrame.from_dict(JSON_time_d)
        JSON_time_df.columns = ['Date and Time', 'Image Name', 'Gantry_x', 'Gantry_y']
        return JSON_time_df

# Converts the gantry coordinates into GPS coordinates (using 'terrautils')
    def GPS_coord (season, date):
        data = JSON.time_df(season, date)
        gantry_x_pos = data['Gantry_x']
        gantry_y_pos = data['Gantry_y']
        GPS_latlon = scanalyzer_to_latlon(gantry_x_pos, gantry_y_pos)
        GPS_df = pd.DataFrame(GPS_latlon)
        GPS_latlon_df = GPS_df.transpose()
        GPS_latlon_df.columns = ['GPS_lat', 'GPS_lon']
        data['GPS_lat'] = GPS_latlon_df['GPS_lat']
        data['GPS_lon'] = GPS_latlon_df['GPS_lon']
        return data

In [25]:
image_gps = JSON.GPS_coord('season_10_yr_2020', '2020-03-03')

In [26]:
lat,lon = image_gps.loc[1]['GPS_lat'], image_gps.loc[1]['GPS_lon']

In [27]:
shp = gpd.read_file('/Users/sebastiancalleja/Desktop/season10_multi_latlon_geno.geojson')

In [28]:
shp

,ID,genotype,geometry
0,MAC Field Scanner Season 10 Range 54 Column 1,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97505 33.07648, -111.9750..."
1,MAC Field Scanner Season 10 Range 54 Column 2,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97505 33.07648, -111.9750..."
2,MAC Field Scanner Season 10 Range 54 Column 3,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97504 33.07648, -111.9750..."
3,MAC Field Scanner Season 10 Range 54 Column 4,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97503 33.07648, -111.9750..."
4,MAC Field Scanner Season 10 Range 54 Column 5,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97503 33.07648, -111.9750..."
...,...,...,...
2587,MAC Field Scanner Season 10 Range 1 Column 44,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97482 33.07456, -111.9748..."
2588,MAC Field Scanner Season 10 Range 1 Column 45,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97481 33.07456, -111.9748..."
2589,MAC Field Scanner Season 10 Range 1 Column 46,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97481 33.07456, -111.9748..."
2590,MAC Field Scanner Season 10 Range 1 Column 47,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97480 33.07456, -111.9748..."


In [65]:
point_list = []
poly_list = []

class Get_plot:
    def to_point (point_data, geom_data):
        for i, row in point_data.iterrows():
            lat = point_data['GPS_lat'].loc[i]
            lon = point_data['GPS_lon'].loc[i]
            points = Point(lon,lat)
            point_list.append(points)
            point_list_df = gpd.GeoDataFrame(point_list)
        return point_list_df

    def poly (point_data, geom_data):
        for i, row in geom_data.iterrows():
            poly_data = geom_data['geometry'].loc[i]
            poly_list.append(poly_data)
            poly_list_df = gpd.GeoDataFrame(poly_list)
        return poly_list_df

In [66]:
GPS_points = Get_plot.to_point(image_gps, shp)

In [67]:
GPS_points.set_index(0)
GPS_points.rename(columns={0: 'Points'}, inplace=True)

In [107]:
data_all = gpd.GeoDataFrame(shp['geometry'])

In [79]:
shp

,ID,genotype,geometry
0,MAC Field Scanner Season 10 Range 54 Column 1,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97505 33.07648, -111.9750..."
1,MAC Field Scanner Season 10 Range 54 Column 2,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97505 33.07648, -111.9750..."
2,MAC Field Scanner Season 10 Range 54 Column 3,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97504 33.07648, -111.9750..."
3,MAC Field Scanner Season 10 Range 54 Column 4,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97503 33.07648, -111.9750..."
4,MAC Field Scanner Season 10 Range 54 Column 5,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97503 33.07648, -111.9750..."
...,...,...,...
2587,MAC Field Scanner Season 10 Range 1 Column 44,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97482 33.07456, -111.9748..."
2588,MAC Field Scanner Season 10 Range 1 Column 45,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97481 33.07456, -111.9748..."
2589,MAC Field Scanner Season 10 Range 1 Column 46,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97481 33.07456, -111.9748..."
2590,MAC Field Scanner Season 10 Range 1 Column 47,Green_Towers_BORDER,"MULTIPOLYGON (((-111.97480 33.07456, -111.9748..."


In [77]:
GPS_points.at[i,"plot"]

,Date and Time,Image Name,Gantry_x,Gantry_y,GPS_lat,GPS_lon
0,03/03/2020 08:45:33,fbf75978-ea6b-4d7c-8d85-9b21835c30fb_metadata....,207.938000,0.011,33.076434,-111.974811
1,03/03/2020 08:45:35,4cb25050-63bf-4669-831f-2c6c7a5c31fa_metadata....,207.938000,0.522,33.076434,-111.974816
2,03/03/2020 08:45:37,e26b63cf-265a-4235-9aed-1b0972c18fcf_metadata....,207.938000,1.011,33.076434,-111.974821
3,03/03/2020 08:45:38,d71b8eef-4830-4839-94cb-a5d3e036684f_metadata....,207.938000,1.515,33.076434,-111.974827
4,03/03/2020 08:45:40,f975830a-9676-49eb-a622-d2b05667ff6c_metadata....,207.938000,2.012,33.076434,-111.974832
...,...,...,...,...,...,...
9265,03/03/2020 13:27:46,cb43835e-f50a-47d9-b3ef-6304f4d34d09_metadata....,3.800988,2.007,33.074593,-111.974831
9266,03/03/2020 13:27:48,2141e0d5-7395-48d3-923b-eebe74586e1a_metadata....,3.800988,1.509,33.074593,-111.974826
9267,03/03/2020 13:27:50,958fa7a8-53de-400d-b7be-2a17bd6ceadc_metadata....,3.800988,1.005,33.074593,-111.974821
9268,03/03/2020 13:27:51,9bbd568e-fc9d-4508-b2aa-ff73c4aca5b6_metadata....,3.800492,0.508,33.074593,-111.974815


In [109]:
def intersection(point):
    for i, row in shp.iterrows():
        #intersection = point.within(data['geometry'].loc[i])
        polygon = gpd.GeoDataFrame(row['geometry'])
        intersection = point.within(polygon)
        if intersection == True:
            #print(intersection)
            #value = shp[shp.index == i]
            plot = row['ID']
            print(plot)
    return plot

In [110]:
GPS_points["plot"] = None
for i, row in GPS_points.iterrows():
    point = row['Points']
    #plot = intersection(point)
    #GPS_points.at[i,"plot"] = plot

In [108]:
data_all

,geometry
0,"MULTIPOLYGON (((-111.97505 33.07648, -111.9750..."
1,"MULTIPOLYGON (((-111.97505 33.07648, -111.9750..."
2,"MULTIPOLYGON (((-111.97504 33.07648, -111.9750..."
3,"MULTIPOLYGON (((-111.97503 33.07648, -111.9750..."
4,"MULTIPOLYGON (((-111.97503 33.07648, -111.9750..."
...,...
2587,"MULTIPOLYGON (((-111.97482 33.07456, -111.9748..."
2588,"MULTIPOLYGON (((-111.97481 33.07456, -111.9748..."
2589,"MULTIPOLYGON (((-111.97481 33.07456, -111.9748..."
2590,"MULTIPOLYGON (((-111.97480 33.07456, -111.9748..."


In [69]:
intersection(data_all[:10])

[]

In [14]:
point_list = []
for i, row in GPS_points.iterrows():
    point = row['Points']
    point_list.append(point)
    point_gdf = gpd.GeoDataFrame(point_list)
print (point_gdf)

                                                 0
0     POINT (-111.9748106187449 33.07643429172416)
1      POINT (-111.974816085573 33.07643428493439)
2     POINT (-111.9748213170386 33.07643427843671)
3     POINT (-111.9748267089786 33.07643427173947)
4     POINT (-111.9748320260305 33.07643426513503)
...                                            ...
9265   POINT (-111.974831342608 33.07459263974853)
9266  POINT (-111.9748260149688 33.07459264636417)
9267  POINT (-111.9748206231411 33.07459265305931)
9268  POINT (-111.9748153061984 33.07459265518653)
9269  POINT (-111.9748101390303 33.07459266160225)

[9270 rows x 1 columns]


In [15]:
poly_list = []
for i, row in data_all.iterrows():
    poly = row['geometry']
    poly_list.append(poly)
    poly_gdf = gpd.GeoDataFrame(poly_list)
print (poly_gdf)

                                                      0
0     POLYGON ((-111.9750491033454 33.07647843197711...
1     POLYGON ((-111.9750454082093 33.07647846087329...
2     POLYGON ((-111.975038384705 33.07647826689856,...
3     POLYGON ((-111.9750344671743 33.07647823530888...
4     POLYGON ((-111.9750273718265 33.07647822856935...
...                                                 ...
2587  POLYGON ((-111.974816679641 33.07455713342119,...
2588  POLYGON ((-111.9748096638381 33.07455762384322...
2589  POLYGON ((-111.9748058292145 33.07455840051821...
2590  POLYGON ((-111.9747988079217 33.07455839318506...
2591  POLYGON ((-111.9747947440462 33.07455848718041...

[2592 rows x 1 columns]
